In [13]:
from selenium import webdriver # Selenium Web自動操作
from selenium.webdriver.common.by import By # HTML要素の検索

from public_gym_search.const import Const

In [14]:
driver = webdriver.Chrome()
# driver.get(Const.GYM_SITE_URL)

In [15]:
global_list = []
for page in range(1,59):
    url = f'https://www.homemate-research-gym.com/list/{ page }'
    driver.get(url)
    
    # 施設名称取得
    elem_target_titles = []
    elems_title = driver.find_elements(By.CLASS_NAME, 'fa_name')
    for elem_title in elems_title:
        elem_target_titles.append(elem_title.text)
    
    # 住所取得
    elem_target_addresses = []
    elems_address = driver.find_elements(By.CLASS_NAME, 'fa_address')
    for index, elem_address in enumerate(elems_address):
        elem_target_addresses.append(elem_address.text)
    
    # リストの中から「アクセス」要素を削除する 
    delete_elem = 'アクセス'
    elem_target_addresses = [item for item in elem_target_addresses if delete_elem not in item]
    
    # 要素の結合
    target_list = [list(items) for items in zip(elem_target_titles, elem_target_addresses)]

    global_list.append(target_list)

In [21]:
# forループを使用して要素を取り出す
for i, element in enumerate(global_list):
    for j, sublist in enumerate(element):
        for k, item in enumerate(sublist):
            print(i)
            print(j)
            print(k)
            print(item)

0
0
0
日本武道館
0
0
1
所在地： 〒102-0091 東京都千代田区北の丸公園２－３
0
1
0
ドルフィンズアリーナ（愛知県体育館）
0
1
1
所在地： 〒460-0032 愛知県名古屋市中区二の丸１－１
0
2
0
講道館
0
2
1
所在地： 〒112-0003 東京都文京区春日１丁目１６－３０
0
3
0
両国国技館
0
3
1
所在地： 〒130-0015 東京都墨田区横網1丁目3番28号
0
4
0
パークアリーナ小牧
0
4
1
所在地： 〒485-0016 愛知県小牧市間々原新田737
0
5
0
春日井市総合体育館
0
5
1
所在地： 〒486-0804 愛知県春日井市鷹来町４１９６－３
0
6
0
マリンメッセ福岡
0
6
1
所在地： 〒812-0031 福岡県福岡市博多区沖浜町７－１
0
7
0
岐阜メモリアルセンターで愛ドーム
0
7
1
所在地： 〒502-0817 岐阜県岐阜市長良福光大野２６７５－２８
0
8
0
モリコロパーク　アイススケート場
0
8
1
所在地： 〒480-1342 愛知県長久手市茨ケ廻間乙1533-1
0
9
0
日本ガイシスポーツプラザ
0
9
1
所在地： 〒457-0833 愛知県名古屋市南区東又兵ヱ町5丁目1番地の16
0
10
0
県立浜山体育館
0
10
1
所在地： 〒699-0722 島根県出雲市大社町北荒木１８６８－１０
0
11
0
福岡国際センター
0
11
1
所在地： 〒812-0021 福岡県福岡市博多区築港本町２－２
0
12
0
北名古屋市健康ドーム
0
12
1
所在地： 〒481-0041 愛知県北名古屋市九之坪笹塚1番地
0
13
0
刈谷市体育館
0
13
1
所在地： 〒448-0838 愛知県刈谷市逢妻町４丁目３２
0
14
0
名古屋市名東スポーツセンター
0
14
1
所在地： 〒465-0067 愛知県名古屋市名東区猪高町高針字勢子坊３０７－１２
0
15
0
東和薬品ＲＡＣＴＡＢドーム（大阪府立門真スポーツセンター）
0
15
1
所在地： 〒571-0015 大阪府門真市三ツ島3丁目7-16
0
16
0
名古屋市千種スポーツセンター
0
16
1
所在地： 〒464-0808 愛知県名古屋市千種区星が丘山手１２１
0
17
0
小田原市総合

In [16]:
from openpyxl import load_workbook

In [17]:
class GymExcelModel(object):
    """ スクレイピング結果→Excel """
    
    def __init__(self):
        self.index = Const.INT_UNSET # for文のindex用
        
        self.facility_name = Const.STRING_EMPTY
        self.prefecture = Const.STRING_EMPTY
        self.municipality = Const.STRING_EMPTY
        self.address = Const.STRING_EMPTY
        self.telephone = None
        self.url = None
        self.training_room_flag = None

In [4]:
class DataCleansing(object):
    """ Data Cleansing """
    def __init__(self, data_list):
        self.data_list = data_list # スクレイピング結果オブジェクト
        self.wb = load_workbook('public_gym_list.xlsx') # Excel Book
        self.ws = self.wb['Sheet1'] # Excel Sheet

    def get_value(self, column, row):
        """ 対象セルの値を取得する """
        if column < 0 or row < 0:
            return ''
        value = str(self.ws.cell(row=row, column=column).value)
        if Const.is_null_or_empty(value):
            return ''
        return value
        
    def set_value(self, column, row, value):
        """ 対象セルに値をセットする """
        self.ws.cell(column=column, row=row).value = value

    def data_cleansing_process(self):
        """ データクレンジング処理 """
        model_list = []
        for row in range(1, 30):
            model = GymExcelModel()
            model_list.append(model)
            
        # Summaryにセットする
        for model in model_list:
            model: GymExcelModel = model
            row = model.index + 3
            
            self.set_value(2, row, model.nationwide_slaughter)
            self.set_value(3, row, model.zennoh_high_price)
            self.set_value(4, row, model.zennoh_middle_price)
            self.set_value(5, row, model.tokyo_high_price)
            self.set_value(6, row, model.tokyo_middle_price)
            self.set_value(7, row, model.tokyo_ordinary_price)
            self.set_value(8, row, model.tokyo_outside_price)
            self.set_value(9, row, model.tokyo_head_count)
        
        self.wb.save('public_gym_list.xlsx')
        
        self.wb.close()

In [44]:
wb = load_workbook('public_gym_list.xlsx') # Excel Book
ws = wb['Sheet1'] # Excel Sheet

In [59]:
model_list = []
for i, element in enumerate(global_list):
    for j, sublist in enumerate(element):
        for k, item in enumerate(sublist):
            model = GymExcelModel()
            if k == 0:
                model.facility_name = item
            if k == 1:
                model.address = item[k+1]
            model_list.append(model)
            print(i, j, k)


print('処理が完了しました')
# wb.save('public_gym_list.xlsx')
# wb.close()

0 0 0
0 0 1
0 1 0
0 1 1
0 2 0
0 2 1
0 3 0
0 3 1
0 4 0
0 4 1
0 5 0
0 5 1
0 6 0
0 6 1
0 7 0
0 7 1
0 8 0
0 8 1
0 9 0
0 9 1
0 10 0
0 10 1
0 11 0
0 11 1
0 12 0
0 12 1
0 13 0
0 13 1
0 14 0
0 14 1
0 15 0
0 15 1
0 16 0
0 16 1
0 17 0
0 17 1
0 18 0
0 18 1
0 19 0
0 19 1
0 20 0
0 20 1
0 21 0
0 21 1
0 22 0
0 22 1
0 23 0
0 23 1
0 24 0
0 24 1
0 25 0
0 25 1
0 26 0
0 26 1
0 27 0
0 27 1
0 28 0
0 28 1
0 29 0
0 29 1
1 0 0
1 0 1
1 1 0
1 1 1
1 2 0
1 2 1
1 3 0
1 3 1
1 4 0
1 4 1
1 5 0
1 5 1
1 6 0
1 6 1
1 7 0
1 7 1
1 8 0
1 8 1
1 9 0
1 9 1
1 10 0
1 10 1
1 11 0
1 11 1
1 12 0
1 12 1
1 13 0
1 13 1
1 14 0
1 14 1
1 15 0
1 15 1
1 16 0
1 16 1
1 17 0
1 17 1
1 18 0
1 18 1
1 19 0
1 19 1
1 20 0
1 20 1
1 21 0
1 21 1
1 22 0
1 22 1
1 23 0
1 23 1
1 24 0
1 24 1
1 25 0
1 25 1
1 26 0
1 26 1
1 27 0
1 27 1
1 28 0
1 28 1
1 29 0
1 29 1
2 0 0
2 0 1
2 1 0
2 1 1
2 2 0
2 2 1
2 3 0
2 3 1
2 4 0
2 4 1
2 5 0
2 5 1
2 6 0
2 6 1
2 7 0
2 7 1
2 8 0
2 8 1
2 9 0
2 9 1
2 10 0
2 10 1
2 11 0
2 11 1
2 12 0
2 12 1
2 13 0
2 13 1
2 14 0
2 14 1
2 15 0
2 1

In [63]:
item

'所在地： 〒080-0330 北海道河東郡音更町雄飛が丘3番地'

In [57]:
model.address

'地'

In [46]:
for i, model in enumerate(model_list):
    model: GymExcelModel = model
    ws.cell(row=i+1, column=1).value = model.facility_name
    ws.cell(row=i+1, column=2).value = model.address
        
wb.save('public_gym_list.xlsx')

wb.close()

In [65]:
wb.close()

In [66]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00928893+48451]
	(No symbol) [0x008BB8A1]
	(No symbol) [0x007C5058]
	(No symbol) [0x007B6F0B]
	(No symbol) [0x007B75CA]
	(No symbol) [0x007B7568]
	(No symbol) [0x007AC0BA]
	(No symbol) [0x007AC917]
	(No symbol) [0x008122F8]
	(No symbol) [0x0080A283]
	(No symbol) [0x007E7E08]
	(No symbol) [0x007E8F2D]
	GetHandleVerifier [0x00B88E3A+2540266]
	GetHandleVerifier [0x00BC8959+2801161]
	GetHandleVerifier [0x00BC295C+2776588]
	GetHandleVerifier [0x009B2280+612144]
	(No symbol) [0x008C4F6C]
	(No symbol) [0x008C11D8]
	(No symbol) [0x008C12BB]
	(No symbol) [0x008B4857]
	BaseThreadInitThunk [0x76A27D59+25]
	RtlInitializeExceptionChain [0x7770B74B+107]
	RtlClearBits [0x7770B6CF+191]
